Importer les librairies

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.master("local").appName("BDA-SQL").getOrCreate()

Créer les tables

In [10]:
t1 = [('07890','Jean Paul Sartre'),
        ('05678','Pierre de Ronsard')]
rdd1 = spark.sparkContext.parallelize(t1) 
author = rdd1.toDF(['aid','name'])
author.show()

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



In [12]:
t2 = [('0001', "L'existentialisme est un humanisme", 'Philosophie'), 
      ('0002', 'Huis clos. Suivi de Les Mouches', 'Philosophie'), 
     ('0003', 'Mignonne allons voir si la rose', 'Poeme'), 
     ('0004', 'Les Amours', 'Poème')]
rdd2 = spark.sparkContext.parallelize(t2)
book = rdd2.toDF(['bid', 'title', 'category'])
book.createOrReplaceTempView('bookSQL')
book.show()

+----+--------------------+-----------+
| bid|               title|   category|
+----+--------------------+-----------+
|0001|L'existentialisme...|Philosophie|
|0002|Huis clos. Suivi ...|Philosophie|
|0003|Mignonne allons v...|      Poeme|
|0004|          Les Amours|      Poème|
+----+--------------------+-----------+



In [13]:
t3 = [('S15','toto','Math'),
        ('S16','popo','Eco'),
        ('S17','fofo','Mécanique')]
rdd3 = spark.sparkContext.parallelize(t3)
student = rdd3.toDF(['sid','sname','dept'])
student.show()

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|     Math|
|S16| popo|      Eco|
|S17| fofo|Mécanique|
+---+-----+---------+



In [14]:
t4 = [('07890','0001'),
        ('07890','0002'),
        ('05678','0003'),
       ('05678','0003')]
rdd4 = spark.sparkContext.parallelize(t4)
write = rdd4.toDF(['aid','bid'])
write.show()

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0003|
+-----+----+



In [15]:
t5 = [('S15','0003','02-01-2020','01-02-2020'),
        ('S15','0002','13-06-2020',None),
        ('S15','0001','13-06-2020','13-10-2020'),
        ('S16','0002','24-01-2020', '24-01-2020'),
        ('S17','0001','12-04-2020','01-07-2020')]
rdd5 = spark.sparkContext.parallelize(t5)
borrow = rdd5.toDF(['sid','bid','checkout-time','return-time'])
borrow.show()

+---+----+-------------+-----------+
|sid| bid|checkout-time|return-time|
+---+----+-------------+-----------+
|S15|0003|   02-01-2020| 01-02-2020|
|S15|0002|   13-06-2020|       null|
|S15|0001|   13-06-2020| 13-10-2020|
|S16|0002|   24-01-2020| 24-01-2020|
|S17|0001|   12-04-2020| 01-07-2020|
+---+----+-------------+-----------+



1-Trouver les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [21]:
#DSL
book\
    .join(borrow,['bid'])\
    .join(student,['sid'])\
    .filter(F.col('sid')=='S15')\
    .select('sid','title')\
    .show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



In [22]:
#SQL
spark.sql("""
    select C.sid, title
    from bookSQL as A
    join borrowSQL as B
    on A.bid = B.bid
    join studentSQL as C
    on B.sid = C.sid
    where B.sid == "S15"
""").show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



2-Trouver les titres de tous les livres qui n'ont jamais été empruntés par un étudiant.

In [23]:
#DSL
book\
    .join(borrow,['bid'], how="full")\
    .join(student,['sid'], how="full")\
    .filter(F.col('sid').isNull())\
    .select('sid','title')\
    .show()

+----+----------+
| sid|     title|
+----+----------+
|null|Les Amours|
+----+----------+



In [24]:
#SQL
spark.sql("""
    select B.sid, title
    from bookSQL as A
    full join borrowSQL as B
    on A.bid = B.bid
    full join studentSQL as C
    on B.sid = C.sid
    where B.sid is NULL
""").show()

+----+----------+
| sid|     title|
+----+----------+
|null|Les Amours|
+----+----------+



3-Trouver tous les étudiants qui ont emprunté le livre bid=’002’

In [25]:
#DSL
book\
    .join(borrow,['bid'])\
    .join(student,['sid'])\
    .filter(F.col('bid')=='0002')\
    .select('bid','sname')\
    .show()

+----+-----+
| bid|sname|
+----+-----+
|0002| popo|
|0002| toto|
+----+-----+



In [26]:
#SQL
spark.sql("""
    select B.bid, sname
    from bookSQL as A
    join borrowSQL as B
    on A.bid = B.bid
    join studentSQL as C
    on B.sid = C.sid
    where B.bid == "0002"
""").show()

+----+-----+
| bid|sname|
+----+-----+
|0002| popo|
|0002| toto|
+----+-----+



4-Trouver les titres de tous les livres empruntés par des étudiants en Mécanique (département Mécanique)

In [27]:
#DSL
book\
    .join(borrow,['bid'])\
    .join(student,['sid'])\
    .filter(F.col('dept')=='Mécanique')\
    .select('dept','title')\
    .show()

+---------+--------------------+
|     dept|               title|
+---------+--------------------+
|Mécanique|L'existentialisme...|
+---------+--------------------+



In [28]:
#SQL
spark.sql("""
    select C.dept, title
    from bookSQL as A
    join borrowSQL as B
    on A.bid = B.bid
    join studentSQL as C
    on B.sid = C.sid
    where C.dept == "Mécanique"
""").show()

+---------+--------------------+
|     dept|               title|
+---------+--------------------+
|Mécanique|L'existentialisme...|
+---------+--------------------+



5-Trouver les étudiants qui n’ont jamais emprunté de livre. 

In [30]:
#DSL
book\
    .join(borrow,['bid'], how="full")\
    .join(student,['sid'], how="full")\
    .filter(F.col('bid').isNull())\
    .select('sid','sname')\
    .show()

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



In [31]:
#SQL
spark.sql("""
    select B.sid, sname
    from bookSQL as A
    full join borrowSQL as B
    on A.bid = B.bid
    full join studentSQL as C
    on B.sid = C.sid 
    where A.bid is NULL
""").show()

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



6- Déterminer l’auteur qui a écrit le plus de livres.

In [32]:
#DSL
nb_livre = write\
            .join(author,['aid'])\
            .groupby('name')\
            .agg(F.count('aid').alias('nb_livre'))
max = nb_livre.collect()[0]['nb_livre']
nb_livre\
    .filter(F.col('nb_livre')==max)\
    .show()

+-----------------+--------+
|             name|nb_livre|
+-----------------+--------+
| Jean Paul Sartre|       2|
|Pierre de Ronsard|       2|
+-----------------+--------+



7- Déterminer les personnes qui n’ont pas encore rendu les livres.

In [35]:
#DSL
borrow\
    .join(student,['sid'], how="full")\
    .filter(F.col('return-time').isNull())\
    .select('sid','sname')\
    .distinct()\
    .show()

+---+-----+
|sid|sname|
+---+-----+
|S15| toto|
+---+-----+



In [36]:
#SQL
spark.sql("""
    select distinct B.sid, sname
    from borrowSQL as A
    join studentSQL as B
    on A.sid = B.sid
    where A.`return-time` is NULL
""").show()

+---+-----+
|sid|sname|
+---+-----+
|S15| toto|
+---+-----+



8- Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0.

In [41]:
#SQL
spark.sql("""
    select *, 
    case
        when (months_between(to_date(`return-time`, 'dd-MM-yyyy'),
                   to_date(`checkout-time`, 'dd-MM-yyyy')) >= 3) then 1
        else 0
    end as `plus que 3 mois`
    from borrowSQL
""").show()

+---+----+-------------+-----------+---------------+
|sid| bid|checkout-time|return-time|plus que 3 mois|
+---+----+-------------+-----------+---------------+
|S15|0003|   02-01-2020| 01-02-2020|              0|
|S15|0002|   13-06-2020|       null|              0|
|S15|0001|   13-06-2020| 13-10-2020|              1|
|S16|0002|   24-01-2020| 24-01-2020|              0|
|S17|0001|   12-04-2020| 01-07-2020|              0|
+---+----+-------------+-----------+---------------+



9-déterminer les livres qui n’ont jamais été empruntés. 

In [42]:
#DSL
book\
    .join(borrow,['bid'], how='full')\
    .filter(F.col('sid').isNull())\
    .select('title')\
    .show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [43]:
#SQL
spark.sql("""
    select A.title
    from bookSQL as A
    full join borrowSQL as B
    on A.bid = B.bid
    where B.sid is NULL
""").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+

